Program ini dibuat untuk melakukan automatisasi transform dan load data dari PostgreSQL ke ElasticSearch,
melakukan validasi data menggunakan Great Expectations, dan melakukan automasi
dengan membuat DAG. Adapun dataset yang dipakai adalah dataset mengenai penjualan sepeda di Eropa
selama tahun 2013 dan 2016.

# Instantiate Data Context

In [51]:
# Create a data context

from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='./')

# Connect to A `Datasource`

In [52]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'csv-milestone-3'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'bike-sales'
path_to_data = 'C:\\Users\\LENOVO\\p2-ftds008-hck-m3-taaratheresia\\P2M3_taara_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# Create an Expectation Suite

In [53]:
# Creat an expectation suite
expectation_suite_name = 'expectation-bike-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,date,day,month,year,customer_age,age_group,customer_gender,country,state,product_category,sub_category,product,order_quantity,unit_cost,unit_price,profit,cost,revenue
0,2013-11-26,26,November,2013,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
1,2015-11-26,26,November,2015,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
2,2014-03-23,23,March,2014,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,23,45,120,1366,1035,2401
3,2016-03-23,23,March,2016,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,20,45,120,1188,900,2088
4,2014-05-15,15,May,2014,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,238,180,418


## Expectations

### Expectation 1 : to be unique

In [54]:
# Expectation 1 : Column `date` must be unique

validator.expect_column_values_to_be_unique('date')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": false,
  "result": {
    "element_count": 9866,
    "unexpected_count": 9860,
    "unexpected_percent": 99.93918508007297,
    "partial_unexpected_list": [
      "2013-11-26",
      "2015-11-26",
      "2014-03-23",
      "2016-03-23",
      "2014-05-15",
      "2016-05-15",
      "2014-05-22",
      "2016-05-22",
      "2014-02-22",
      "2016-02-22",
      "2013-07-30",
      "2015-07-30",
      "2013-07-15",
      "2015-07-15",
      "2013-08-02",
      "2015-08-02",
      "2013-09-02",
      "2015-09-02",
      "2014-01-22",
      "2016-01-22"
    ],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 99.93918508007297,
    "unexpected_percent_nonmissing": 99.93918508007297
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**PENJELASAN:**

Pada dataset ini, tidak ada kolom yang memiliki unique values. Contoh nya seperti kolom `date` karena transaksi penjualan sepeda bisa terjadi beberapa kali dalam 1 hari yang sama di berbagai negara.

### Expectation 2 : to be between min_value and max_value

In [55]:
# Expectation 2 : Column `unit_cost` must be less than $ 100

validator.expect_column_values_to_be_between(
    column='unit_cost', min_value=0, max_value=100
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9866,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**PENJELASAN:**

- Dalam banyak bisnis, kisaran biaya unit yang masuk akal adalah dari 0 hingga 100. Ini mencakup biaya pembuatan, biaya bahan baku, atau biaya lain yang dapat diukur dalam rentang tersebut.
- Memiliki kebijakan atau praktik yang membatasi biaya unit produk hingga 100 atau kurang. Ini dilakukan untuk menjaga harga jual tetap kompetitif atau mengelola keuntungan.

### Expectation 3 : to be in set

In [56]:
# Expectation 3 : Column `customer_gender` must contain one of the following 2 things :
# M = Male
# F = Female

validator.expect_column_values_to_be_in_set(
    "customer_gender",
    ['M','F']
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9866,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**PENJELASAN:**

Pada kolom `customer_gender` harus berisi Male (M) dan Female (F).

### Expectation 4 : to be in type list

In [57]:
# Expectation 4 : Column `customer_age` must in form of int64

validator.expect_column_values_to_be_in_type_list(
    'customer_age', 
    ['int64']
    )

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**PENJELASAN:**

Pada kolom `customer_age` harus berisi tipe data int64, karena usia adalah bilangan bulat non-negatif dan tidak boleh memiliki fraksional atau nilai desimal.

### Expectation 5 : lengths to be between

In [58]:
# Expectation 5 : The length of the `product` column value must be less than 25

validator.expect_column_value_lengths_to_be_between(
    column='product', 
    min_value=0, 
    max_value=25
)

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9866,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**PENJELASAN:**

- Nama produk atau deskripsi produk umumnya tidak melebihi 25 karakter, dan ini adalah batasan yang telah ditetapkan berdasarkan praktik bisnis yang berlaku.
-  Ada pembatasan pada tampilan atau cetakan laporan yang membatasi panjang teks yang dapat ditampilkan. Dengan membatasi panjang produk hingga 25 karakter, ini memungkinkan produk tetap dapat ditampilkan dengan benar dalam konteks tertentu.

### Expectation 6 : to match regex

In [72]:
# Expectation 6 : The 'product_category' column will match values ​​consisting of only capital letters from A to Z.

validator.expect_column_values_to_match_regex(
    column='product_category', 
    regex=r'[A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|V|W|X|Y|Z]+'
    )

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9866,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**PENJELASAN:**

Pola regex tersebut sesuai jika `product_category` diharapkan hanya berisi nama produk yang diawali dengan huruf kapital tanpa karakter lainnya.

### Expectation 7 : to not be null

In [74]:
# Expectation 7 : Column `order_quantity` can not contain missing values

validator.expect_column_values_to_not_be_null('order_quantity')

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9866,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**PENJELASAN:**

'order_quantity' adalah elemen penting dari setiap pesanan atau transaksi, dan setiap pesanan harus memiliki jumlah yang dipesan. Dalam kasus ini, data yang hilang dalam kolom 'order_quantity' bisa sangat mengganggu dan berdampak pada akurasi analisis, pelacakan inventaris, dan pengelolaan pesanan.

In [75]:
# Save into Expectation Suite

validator.save_expectation_suite(discard_failed_expectations=False)

## Checkpoint

In [76]:
# Create a checkpoint

checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

In [77]:
# Run a checkpoint

checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/68 [00:00<?, ?it/s]

## Data Docs

In [78]:
# Build data docs

context.build_data_docs()

{'local_site': 'file://c:\\Users\\LENOVO\\p2-ftds008-hck-m3-taaratheresia\\gx\\uncommitted/data_docs/local_site/index.html'}